1. 
First need to read in the weekly returns for the 10 specified stocks from the period Jan 2005 - Dec 2009.
Stocks are Wells Fargo, JP Morgan, Apple, Microsoft, Google, IBM, Walmart, AIG, and GE.

In [28]:
#Need to run this command at anaconda prompt: "$ pip3 install yfinance"
import yfinance as yf
import pandas as pd
from matplotlib import pyplot as plt
import math
import numpy as np


In [4]:
data = yf.download(
        tickers = 'wfc jpm aapl msft googl ibm wmt aig ge ', 
        start = "2005-01-01", 
        end = "2008-12-31",
        interval = "1wk",
        group_by = 'ticker',
        threads = True)

[*********************100%***********************]  9 of 9 completed


Now lets turn this multi-index df into a regular df and looking only at AdjClose which we'll use to calculate 
the logreturn values weekly. Also, periodic NA values result from yfinance throwing in random non week ending datapoints, probably resulting from choosing Jan 1st 2005 as the starting point which was a Saturday. So, lets drop those and then calculate log returns. 

In [61]:
AdjClose = data.iloc[:, data.columns.get_level_values(1) == 'Adj Close']
processed = AdjClose.dropna()
processed.columns = processed.columns.droplevel(1)  #turns columns from multi-index to normal index
logRet = np.log(processed)
logRet = logRet.diff()
logRet = logRet.dropna()
logRet.head()

,MSFT,AIG,AAPL,GE,JPM,IBM,WMT,WFC,GOOGL
Date,,,,,,,,,
2005-01-08,-0.020839,-0.021539,0.013625,-0.013423,-0.006761,-0.017697,0.000000,-0.024094,0.031083
2005-01-15,-0.018157,-0.000302,0.004122,-0.011040,-0.025718,-0.018447,-0.018318,-0.009436,-0.060237
2005-01-22,0.020452,-0.003940,0.048324,0.017495,0.004063,0.005506,-0.011001,0.006631,0.010882
2005-01-29,0.005334,0.024893,0.063626,0.013889,0.018477,0.017290,0.019455,0.007572,0.071071
2005-02-05,-0.013387,0.079678,0.029618,-0.000552,-0.005587,-0.012886,-0.025769,-0.003416,-0.086638


2. 
Conduct the NPEB analysis on the individual returns in order to generate the return distribution as well as the covariance relationships.

3. 
Calculate the Expected Shortfall (cVARa) estimates for the data at each time point

4. 
Calculate the optimal portfolio weights via grid search, optimization

5. 
Using these dynamic portfolio weights, model the cumulative returns for the period and compare them to just holding the market basket.